# **Практическая работа №5. Геоанализ экологических факторов городской среды с использованием технологии Apache Sedona**




### **Цель работы**
















Освоение методов пространственного анализа распределенных данных для оценки экологической обстановки городской территории с использованием фреймворка Apache Sedona, с последующей постообработкой и обучением моделей машинного обучения.

### **Задачи**


1. Освоить основные инструменты и методы загрузки экологических геоданных
2. Реализовать анализ пространственного распределения экологических факторов на городской территории
3. Применить технологии распределенных вычислений для обработки больших объемов пространственных данных
4. Провести кластерный анализ и классификацию территорий по экологическим показателям

### **Теоретическая часть**


Современный геоэкологический анализ требует обработки значительных объемов пространственно-распределенных данных. Apache Sedona представляет собой высокопроизводительный фреймворк для выполнения пространственных запросов и анализа с использованием распределенных вычислений, что позволяет эффективно обрабатывать большие объемы пространственных данных.

### **Этапы работы**


#### **1. Определение области исследования и подготовка данных**


- Выберите городскую территорию для анализа экологической обстановки
- Используя API OpenStreetMap и другие открытые источники, загрузите данные следующих категорий:
  - Источники загрязнения (промышленные предприятия, мусоропереработка, ТЭЦ)
  - Зеленые насаждения (парки, скверы, лесопарковые зоны)
  - Водные объекты (реки, водоемы)
  - Автомагистрали (категории дорог с интенсивным движением)
  - Административные районы города
- Преобразуйте полученные данные в форматы GeoJSON и CSV для дальнейшей обработки

#Выбранная городская территория - город Хабаровск

In [2]:
%%capture
# Установка библиотек для обработки геоданных и визуализации
!pip install scikit-learn geopandas h3pandas h3~=3.0 leafmap mapclassify matplotlib streamlit osmnx openrouteservice polyline -q

In [4]:
# Импорт основных библиотек
import geopandas as gpd
import pandas as pd
import numpy as np
import h3
import h3pandas
import leafmap
from shapely.geometry import box
from sklearn.preprocessing import MinMaxScaler

In [5]:
m = leafmap.Map(center=[48.4827, 135.0838], zoom=10, draw_control=True)
m

Map(center=[48.4827, 135.0838], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

In [6]:
bbox = m.user_roi_bounds()

if bbox:
    print(f"Выбранный bounding box: {bbox}")
else:
    bbox = []

Выбранный bounding box: [135.0145, 48.3403, 135.178, 48.5402]


In [7]:
import osmnx as ox
import warnings
import os

# Функция для загрузки данных из OSM по выбранной области
def load_environmental_osm_data(bbox):
    west, south, east, north = bbox

    # Теги для поиска
    tags = {
        'industrial': {'landuse': 'industrial'},  # Источники загрязнения
        'parks': {'leisure': 'park'},             # Зеленые насаждения
        'water': {'natural': 'water'},            # Водные объекты
        'highways': {'highway': True},            # Автомагистрали
        'districts': {'boundary': 'administrative', 'admin_level': '9'}  # Районы города
    }

    # Загрузка данных с обработкой исключений
    data = {}
    for key, tag in tags.items():
        print(f"\nЗагрузка {key}...")
        try:
            # Используем позиционные аргументы
            gdf = ox.features_from_bbox(bbox, tag)
            data[key] = gdf
            print(f"Успешно загружено объектов: {len(gdf)}")
        except Exception as e:
            print(f"Ошибка при загрузке {key}: {e}")
            data[key] = gpd.GeoDataFrame(columns=['geometry'], crs="EPSG:4326")

    # Фильтруем уровень границ,
    if 'districts' in data and not data['districts'].empty:
        if 'admin_level' in data['districts'].columns:
            before = len(data['districts'])
            data['districts'] = data['districts'][data['districts']['admin_level'].isin(['9', '10'])].copy()
            after = len(data['districts'])
            print(f"Фильтрация районов: было {before}, осталось {after}")

    # Очистка столбцов с более чем 50% пропущенных значений

    for key in data.keys():
        if not data[key].empty:
            threshold = len(data[key]) * 0.6
            data[key] = data[key].dropna(axis=1, thresh=threshold)
            print(f"Очистка столбцов для {key} завершена. Осталось столбцов: {len(data[key].columns)}")

    return data

env_data = load_environmental_osm_data(bbox)


Загрузка industrial...
Успешно загружено объектов: 171

Загрузка parks...
Успешно загружено объектов: 97

Загрузка water...
Успешно загружено объектов: 288

Загрузка highways...
Успешно загружено объектов: 16426

Загрузка districts...
Успешно загружено объектов: 45
Фильтрация районов: было 45, осталось 5
Очистка столбцов для industrial завершена. Осталось столбцов: 2
Очистка столбцов для parks завершена. Осталось столбцов: 2
Очистка столбцов для water завершена. Осталось столбцов: 2
Очистка столбцов для highways завершена. Осталось столбцов: 2
Очистка столбцов для districts завершена. Осталось столбцов: 8


In [8]:
# Функция для сохранения данных в формате GeoJSON
def save_to_geojson(data, output_dir="osm_data"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for key, gdf in data.items():
        if not gdf.empty:
            # Конвертируем в WGS84 (EPSG:4326) для GeoJSON
            gdf_wgs84 = gdf.to_crs("EPSG:4326")
            output_file = os.path.join(output_dir, f"{key}.geojson")
            gdf_wgs84.to_file(output_file, driver="GeoJSON")
            print(f"Данные '{key}' сохранены в {output_file}")
        else:
            print(f"Нет данных для '{key}', пропускаем сохранение")

# Сохраняем в GeoJSON
save_to_geojson(env_data)

Данные 'industrial' сохранены в osm_data/industrial.geojson
Данные 'parks' сохранены в osm_data/parks.geojson
Данные 'water' сохранены в osm_data/water.geojson
Данные 'highways' сохранены в osm_data/highways.geojson
Данные 'districts' сохранены в osm_data/districts.geojson


In [9]:
# Функция для сохранения данных в формате CSV
def save_to_csv(data, output_dir="osm_data"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for key, gdf in data.items():
        if not gdf.empty:
            output_file = os.path.join(output_dir, f"{key}.csv")
            gdf_copy = gdf.copy()

            # Приведение к WGS84 и конвертация геометрии в WKT
            if gdf_copy.crs and gdf_copy.crs != "EPSG:4326":
                gdf_copy = gdf_copy.to_crs("EPSG:4326")
            gdf_copy['geometry'] = gdf_copy['geometry'].apply(lambda g: g.wkt if g else None)

            gdf_copy.to_csv(output_file, index=False)
            print(f"Данные '{key}' сохранены в {output_file}")
        else:
            print(f"Нет данных для '{key}', пропускаем сохранение")

# Сохраняем в CSV
save_to_csv(env_data)

Данные 'industrial' сохранены в osm_data/industrial.csv
Данные 'parks' сохранены в osm_data/parks.csv
Данные 'water' сохранены в osm_data/water.csv
Данные 'highways' сохранены в osm_data/highways.csv
Данные 'districts' сохранены в osm_data/districts.csv


#### **2. Настройка среды распределенных вычислений**


- Установите и настройте среду Apache Sedona
- Загрузите подготовленные геоданные в формате Sedona DataFrame
- Создайте временные представления для выполнения пространственных SQL-запросов

In [10]:
%%capture
# Установка Sedona и вспомогательных библиотек
!pip install apache-sedona[spark]
!pip install findspark
!pip install -I shapely==1.8
!pip install geopandas==0.13.2

# Библиотеки для визуализации
!pip install pydeck
!pip install keplergl

In [11]:
from pyspark.sql import SparkSession
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, LongType
from shapely.geometry import Point, Polygon
from sedona.spark import *
from sedona import *
from sedona.core.geom.envelope import Envelope
from pyspark.sql.functions import expr, col, log10, when, lit
import findspark

In [12]:
# Инициализация Spark
findspark.init()

# Настройка конфигурации Sedona
config = SedonaContext.builder() \
    .config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-3.4_2.12:1.5.3,'
           'org.datasyslab:geotools-wrapper:1.5.3-28.2,'
           'uk.co.gresearch.spark:spark-extension_2.12:2.11.0-3.4') \
    .config('spark.jars.repositories', 'https://artifacts.unidata.ucar.edu/repository/unidata-all') \
    .getOrCreate()

# Создание контекста Sedona
sedona = SedonaContext.create(config)
sc = sedona.sparkContext

# Установка кодировки для корректной работы с текстовыми данными
sc.setSystemProperty("sedona.global.charset", "utf8")

In [14]:
# Загрузка геоданных в Sedona
# Пути к файлам
industrial_path = "osm_data/industrial.csv"
parks_path = "osm_data/parks.csv"
water_path = "osm_data/water.csv"
highways_path = "osm_data/highways.csv"
districts_path = "osm_data/districts.csv"

# Функция для загрузки CSV в Sedona DataFrame
def csv_to_sedona_df(csv_path, sedona_context, geometry_column="geometry"):
    # Чтение CSV файла через Spark
    df = sedona_context.read.option("header", "true").option("inferSchema", "true").csv(csv_path)

    # Проверяем наличие столбца с геометрией
    if geometry_column in df.columns:
        # Заменяем пустые строки на null
        df = df.withColumn(
            geometry_column,
            when(col(geometry_column) == "", None).otherwise(col(geometry_column))
        )

        # Преобразуем WKT-строку в геометрию Sedona
        df = df.withColumn("geometry", expr(f"ST_GeomFromWKT({geometry_column})"))

        # Если имя столбца отличается от 'geometry', удаляем исходный столбец
        if geometry_column != "geometry":
            df = df.drop(geometry_column)

        return df

# Загрузка данных из CSV-файлов
industrial_df = csv_to_sedona_df(industrial_path, sedona)
parks_df = csv_to_sedona_df(parks_path, sedona)
water_df = csv_to_sedona_df(water_path, sedona)
highways_df = csv_to_sedona_df(highways_path, sedona)
districts_df = csv_to_sedona_df(districts_path, sedona)

# Создание временных представлений для SQL-запросов
industrial_df.createOrReplaceTempView("industrial")
parks_df.createOrReplaceTempView("parks")
water_df.createOrReplaceTempView("water")
highways_df.createOrReplaceTempView("highways")
districts_df.createOrReplaceTempView("districts")

In [15]:
# Проверка
districts_df.show()

+--------------------+-----------+--------------+--------------------+---------+--------+--------+---------+
|            geometry|admin_level|      boundary|                name|   source|wikidata|    type|addr:city|
+--------------------+-----------+--------------+--------------------+---------+--------+--------+---------+
|POLYGON ((135.050...|          9|administrative|Железнодорожный р...|Росреестр|Q4178970|boundary|Хабаровск|
|POLYGON ((134.897...|          9|administrative|Индустриальный район|Росреестр|Q4200937|boundary|Хабаровск|
|POLYGON ((134.937...|          9|administrative|     Кировский район|Росреестр|Q4221688|boundary|Хабаровск|
|POLYGON ((134.980...|          9|administrative|Краснофлотский район|Росреестр|Q4239147|boundary|Хабаровск|
|POLYGON ((134.901...|          9|administrative|   Центральный район|Росреестр|Q4504361|boundary|Хабаровск|
+--------------------+-----------+--------------+--------------------+---------+--------+--------+---------+



#### **3. Базовый пространственный анализ экологических факторов**


- Рассчитайте для каждого административного района:
  - Количество и плотность источников загрязнения
  - Процент покрытия зелеными насаждениями
  - Протяженность водных объектов
  - Плотность автомагистралей
- Выполните нормализацию полученных показателей
- Сформируйте свой собственный интегральный индекс экологического благополучия территории
- Визуализируйте результаты на карте города

In [16]:
# Количество и плотность источников загрязнения
pollution_df = sedona.sql("""
SELECT
    d.geometry AS district_geom,
    COUNT(i.geometry) AS pollution_sources_count,
    COUNT(i.geometry) / ST_Area(d.geometry) AS pollution_density
FROM districts d
LEFT JOIN industrial i ON ST_Contains(d.geometry, i.geometry)
GROUP BY d.geometry
""")

pollution_df.createOrReplaceTempView("pollution_stats")
pollution_df.show()

+--------------------+-----------------------+-----------------+
|       district_geom|pollution_sources_count|pollution_density|
+--------------------+-----------------------+-----------------+
|POLYGON ((134.980...|                      6|694.5203922392393|
|POLYGON ((134.937...|                     16|3496.212252079047|
|POLYGON ((135.050...|                     73|6045.525084419313|
|POLYGON ((134.901...|                      5|786.8703240560741|
|POLYGON ((134.897...|                     52|3340.920953515586|
+--------------------+-----------------------+-----------------+



In [17]:
# Процент покрытия зелеными насаждениями
green_cover_df = sedona.sql("""
SELECT
    d.geometry AS district_geom,
    SUM(ST_Area(p.geometry)) AS green_area,
    ST_Area(d.geometry) AS district_area,
    SUM(ST_Area(p.geometry)) / ST_Area(d.geometry) * 100 AS green_percent
FROM districts d
LEFT JOIN parks p ON ST_Intersects(d.geometry, p.geometry)
GROUP BY d.geometry
""")

green_cover_df.createOrReplaceTempView("green_cover_stats")
green_cover_df.show()

+--------------------+--------------------+--------------------+-------------------+
|       district_geom|          green_area|       district_area|      green_percent|
+--------------------+--------------------+--------------------+-------------------+
|POLYGON ((134.980...|3.203727027996886E-5|0.008639055191245124|0.37084229201864166|
|POLYGON ((134.937...|2.853298645000269...| 0.00457638119381496| 0.6234836050931555|
|POLYGON ((135.050...| 2.29891403049824E-5|0.012075047077074831| 0.1903855128534331|
|POLYGON ((134.901...|9.994690748993317E-5| 0.00635428716414992| 1.5729051097001234|
|POLYGON ((134.897...|3.790082619503697E-5| 0.01556457058503028|0.24350704690490657|
+--------------------+--------------------+--------------------+-------------------+



In [18]:
# Протяженность водных объектов
water_length_df = sedona.sql("""
SELECT
    d.geometry AS district_geom,
    SUM(ST_Length(w.geometry)) AS water_length
FROM districts d
LEFT JOIN water w ON ST_Intersects(d.geometry, w.geometry)
GROUP BY d.geometry
""")

water_length_df.createOrReplaceTempView("water_stats")
water_length_df.show()

+--------------------+------------------+
|       district_geom|      water_length|
+--------------------+------------------+
|POLYGON ((134.980...| 2.584486422615817|
|POLYGON ((134.937...|2.6204504292996162|
|POLYGON ((135.050...|0.3475228130669866|
|POLYGON ((134.901...| 6.792482549244611|
|POLYGON ((134.897...| 7.149797108267741|
+--------------------+------------------+



In [19]:
# Плотность автомагистралей
highway_density_df = sedona.sql("""
SELECT
    d.geometry AS district_geom,
    SUM(ST_Length(h.geometry)) AS total_highway_length,
    ST_Area(d.geometry) AS district_area,
    SUM(ST_Length(h.geometry)) / ST_Area(d.geometry) AS highway_density
FROM districts d
LEFT JOIN highways h ON ST_Intersects(d.geometry, h.geometry)
GROUP BY d.geometry
""")

highway_density_df.createOrReplaceTempView("highway_stats")
highway_density_df.show()

+--------------------+--------------------+--------------------+-----------------+
|       district_geom|total_highway_length|       district_area|  highway_density|
+--------------------+--------------------+--------------------+-----------------+
|POLYGON ((134.980...|  1.3080115645432653|0.008639055191245124|151.4067841433416|
|POLYGON ((134.937...|  1.8270491204698034| 0.00457638119381496|399.2344700085484|
|POLYGON ((135.050...|  10.006758528378882|0.012075047077074831|828.7138314663208|
|POLYGON ((134.901...|  3.5270407842685896| 0.00635428716414992| 555.064744975283|
|POLYGON ((134.897...|  10.324069047798213| 0.01556457058503028|663.3057424486683|
+--------------------+--------------------+--------------------+-----------------+



In [20]:
# Объединяем все метрики в один DataFrame
analysis_df = sedona.sql("""
SELECT
    p.district_geom AS district_geom,

    COALESCE(p.pollution_sources_count, 0) AS pollution_sources_count,
    COALESCE(p.pollution_density, 0.0) AS pollution_density,

    COALESCE(g.green_area, 0.0) AS green_area,
    COALESCE(g.green_percent, 0.0) AS green_percent,

    COALESCE(w.water_length, 0.0) AS water_length,

    COALESCE(h.total_highway_length, 0.0) AS total_highway_length,
    COALESCE(h.highway_density, 0.0) AS highway_density

FROM pollution_stats p
LEFT JOIN green_cover_stats g ON ST_Equals(p.district_geom, g.district_geom)
LEFT JOIN water_stats w ON ST_Equals(p.district_geom, w.district_geom)
LEFT JOIN highway_stats h ON ST_Equals(p.district_geom, h.district_geom)
""")

analysis_df.createOrReplaceTempView("analysis")

In [21]:
analysis_df.show()

+--------------------+-----------------------+-----------------+--------------------+-------------------+------------------+--------------------+-----------------+
|       district_geom|pollution_sources_count|pollution_density|          green_area|      green_percent|      water_length|total_highway_length|  highway_density|
+--------------------+-----------------------+-----------------+--------------------+-------------------+------------------+--------------------+-----------------+
|POLYGON ((134.980...|                      6|694.5203922392393|3.203727027996886E-5|0.37084229201864166| 2.584486422615817|  1.3080115645432653|151.4067841433416|
|POLYGON ((134.937...|                     16|3496.212252079047|2.853298645000269...| 0.6234836050931555|2.6204504292996162|  1.8270491204698034|399.2344700085484|
|POLYGON ((135.050...|                     73|6045.525084419313| 2.29891403049824E-5| 0.1903855128534331|0.3475228130669866|  10.006758528378882|828.7138314663208|
|POLYGON ((134.9

In [22]:
# Нормализация данных
from pyspark.ml.feature import MinMaxScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.functions import vector_to_array

# Подготавливаем данные для нормализации
assembler = VectorAssembler(
    inputCols=["pollution_sources_count", "pollution_density", "green_area", "green_percent", "water_length", "total_highway_length", "highway_density"],
    outputCol="features"
)
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
pipeline = Pipeline(stages=[assembler, scaler])

# Применяем нормализацию
scaler_model = pipeline.fit(analysis_df)
scaled_data = scaler_model.transform(analysis_df)

# Преобразуем вектор в отдельные столбцы и вычисляем индекс
scaled_df = scaled_data.withColumn("scaled_array", vector_to_array(col("scaled_features")))
scaled_df = scaled_df.select(
    col("district_geom"),
    col("scaled_array")[0].alias("pollution_sources_count_norm"),
    col("scaled_array")[1].alias("pollution_density_norm"),
    col("scaled_array")[2].alias("green_area_norm"),
    col("scaled_array")[3].alias("green_percent_norm"),
    col("scaled_array")[4].alias("water_length_norm"),
    col("scaled_array")[5].alias("total_highway_length_norm"),
    col("scaled_array")[6].alias("highway_density_norm")
)

scaled_df.createOrReplaceTempView("scaled_analysis")

In [23]:
w_pollution = 0.7
w_green = 1.2
w_water = 0.9
w_highways = 0.3

weighted_df = scaled_df.select(
    "district_geom",
    col("pollution_density_norm"),
    col("green_area_norm"),
    col("water_length_norm"),
    col("highway_density_norm")
).withColumn(
    "eco_index",
    w_green * col("green_area_norm") +
    w_water * col("water_length_norm") -
    w_pollution * col("pollution_density_norm") -
    w_highways * col("highway_density_norm")
)

In [24]:
# Фильтрация полигонов для визуализации
weighted_polygons = weighted_df.filter(
    expr("ST_GeometryType(district_geom) IN ('ST_Polygon', 'ST_MultiPolygon')")
)

weighted_polygons.createOrReplaceTempView("eco_weighted_index_polygons")

# Визуализация с помощью SedonaPyDeck
eco_index_map = SedonaPyDeck.create_choropleth_map(
    df=weighted_polygons,
    plot_col='eco_index',
    map_style='light'
)

# Отображение карты
eco_index_map

<IPython.core.display.Javascript object>

{
  "initialViewState": {
    "latitude": 48.500004419999996,
    "longitude": 134.95356856,
    "zoom": 11
  },
  "layers": [
    {
      "@@type": "GeoJsonLayer",
      "autoHighlight": true,
      "data": [
        {
          "coordinate_array_sedona": [
            134.9803613,
            48.5765601
          ],
          "eco_index": 0.437056925456504,
          "geometry": {
            "coordinates": [
              [
                [
                  134.9803613,
                  48.5765601
                ],
                [
                  134.9805541,
                  48.5766783
                ],
                [
                  134.9814524,
                  48.5772868
                ],
                [
                  134.9822132,
                  48.5779796
                ],
                [
                  134.9831612,
                  48.5786301
                ],
                [
                  134.9838384,
                  48.5789464
                ],
                [
                  134.9847972,
                  48.5793648
                ],
                [
                  134.9858916,
                  48.5798723
                ],
                [
                  134.9869859,
                  48.5803266
                ],
                [
                  134.9880993,
                  48.5807766
                ],
                [
                  134.9892826,
                  48.5813231
                ],
                [
                  134.9904146,
                  48.5819634
                ],
                [
                  134.9913346,
                  48.5822788
                ],
                [
                  134.9930423,
                  48.582644
                ],
                [
                  134.9964588,
                  48.5831939
                ],
                [
                  134.9998758,
                  48.5837405
                ],
                [
                  135.0029369,
                  48.5842336
                ],
                [
                  135.0068695,
                  48.5848679
                ],
                [
                  135.0091423,
                  48.5852397
                ],
                [
                  135.0111855,
                  48.5855732
                ],
                [
                  135.0142354,
                  48.5860697
                ],
                [
                  135.0168263,
                  48.586492
                ],
                [
                  135.0198452,
                  48.586985
                ],
                [
                  135.0233807,
                  48.587568
                ],
                [
                  135.025685,
                  48.5879505
                ],
                [
                  135.0278242,
                  48.588305
                ],
                [
                  135.0297733,
                  48.5886275
                ],
                [
                  135.0308273,
                  48.5888016
                ],
                [
                  135.0315353,
                  48.5889631
                ],
                [
                  135.0315401,
                  48.5890695
                ],
                [
                  135.031573,
                  48.589179
                ],
                [
                  135.0316031,
                  48.5892762
                ],
                [
                  135.0316178,
                  48.5892913
                ],
                [
                  135.0316329,
                  48.5893013
                ],
                [
                  135.0316546,
                  48.5893493
                ],
                [
                  135.0317132,
                  48.5893453
                ],
                [
                  135.0317771,
                  48.5894955
       

#### **4. Детализированный анализ с использованием геосетки H3**


- Создайте регулярную геопространственную сетку H3 для выбранной территории
- Определите буферные зоны влияния для различных экологических факторов:
  - 2 км для промышленных предприятий
  - 500 м для автомагистралей
  - 1 км для зеленых зон и водоемов
- Проведите расчет экологических показателей для каждой ячейки H3
- Нормализуйте и интегрируйте полученные показатели
- Постройте карту экологического благополучия с высоким пространственным разрешением

In [25]:
from pyspark.sql.functions import expr, col, lit, udf
from pyspark.sql.types import ArrayType, StringType
import h3
from shapely import wkt
from shapely.geometry import Polygon

In [26]:
# Создание полигона из bbox
bbox_polygon_sql = f"""
    SELECT ST_GeomFromWKT('POLYGON(({bbox[0]} {bbox[1]}, {bbox[0]} {bbox[3]},
    {bbox[2]} {bbox[3]}, {bbox[2]} {bbox[1]}, {bbox[0]} {bbox[1]}))') AS geometry
"""
bbox_polygon = sedona.sql(bbox_polygon_sql)
bbox_polygon.createOrReplaceTempView("bbox_polygon")

In [27]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, DoubleType

# Определение UDF-функций для работы с H3
@udf(returnType=ArrayType(StringType()))
def polygon_to_h3(geom_wkt, resolution):
    geom = wkt.loads(geom_wkt)
    geojson_geom = geom.__geo_interface__
    return list(h3.polyfill_geojson(geojson_geom, resolution))

# Преобразование H3-индекса в полигон WKT
@udf(returnType=StringType())
def h3_to_polygon_wkt(h3_index):
    boundary = h3.h3_to_geo_boundary(h3_index, geo_json=True)
    polygon = Polygon(boundary)
    return polygon.wkt

In [28]:
# Генерация H3-ячеек для выбранной области
resolution = 9  # Уровень детализации сетки H3
h3_indices_df = bbox_polygon.select(
    polygon_to_h3(expr("ST_AsText(geometry)"), lit(resolution)).alias("h3_indices")
)
h3_exploded_df = h3_indices_df.selectExpr("explode(h3_indices) as h3_index")

In [29]:
# Преобразование H3-индексов в геометрии
h3_gdf = h3_exploded_df.select(
    col("h3_index"),
    h3_to_polygon_wkt(col("h3_index")).alias("geometry_wkt")
).withColumn(
    "geometry", expr("ST_GeomFromWKT(geometry_wkt)")
).drop("geometry_wkt")

h3_gdf.createOrReplaceTempView("h3_cells")

In [30]:
h3_gdf.show()

+---------------+--------------------+
|       h3_index|            geometry|
+---------------+--------------------+
|8914d64a9cbffff|POLYGON ((135.068...|
|8914d64f1cfffff|POLYGON ((135.157...|
|8914d64e573ffff|POLYGON ((135.170...|
|8914d64ad7bffff|POLYGON ((135.056...|
|8914d648abbffff|POLYGON ((135.106...|
|892eda4cb47ffff|POLYGON ((135.105...|
|8914d64aa6bffff|POLYGON ((135.059...|
|8914d64f6a7ffff|POLYGON ((135.116...|
|8914d64f147ffff|POLYGON ((135.165...|
|8914d641a2fffff|POLYGON ((135.097...|
|8914d648b73ffff|POLYGON ((135.138...|
|8914d64e247ffff|POLYGON ((135.165...|
|8914d64a9c3ffff|POLYGON ((135.068...|
|8914d649a87ffff|POLYGON ((135.073...|
|8914d641c7bffff|POLYGON ((135.057...|
|8914d64f3a3ffff|POLYGON ((135.168...|
|8914d64a95bffff|POLYGON ((135.072...|
|8914d649c33ffff|POLYGON ((135.058...|
|8914d648873ffff|POLYGON ((135.111...|
|8914d64e07bffff|POLYGON ((135.170...|
+---------------+--------------------+
only showing top 20 rows



In [31]:
from pyspark.sql.functions import expr

# Создание буферов разного размера для анализа
h3_buffers_df = sedona.sql("""
    SELECT
        h3_index,
        geometry,
        -- Трансформация в систему координат EPSG:3857 для точных буферов
        ST_Transform(geometry, 'EPSG:4326', 'EPSG:3857') AS geometry_3857,

        -- Буфер 2 км для промышленных предприятий
        ST_Buffer(ST_Transform(geometry, 'EPSG:4326', 'EPSG:3857'), 2000) AS industrial_buffer_2km,

        -- Буфер 500 м для автомагистралей
        ST_Buffer(ST_Transform(geometry, 'EPSG:4326', 'EPSG:3857'), 500) AS highway_buffer_500m,

        -- Буфер 1 км для зеленых зон и водоемов
        ST_Buffer(ST_Transform(geometry, 'EPSG:4326', 'EPSG:3857'), 1000) AS green_water_buffer_1km

    FROM h3_cells
""")
h3_buffers_df.createOrReplaceTempView("h3_buffers")

In [32]:
# 3. Проверка буферов (просмотр первых строк для проверки)
h3_buffers_df.show()

+---------------+--------------------+--------------------+---------------------+--------------------+----------------------+
|       h3_index|            geometry|       geometry_3857|industrial_buffer_2km| highway_buffer_500m|green_water_buffer_1km|
+---------------+--------------------+--------------------+---------------------+--------------------+----------------------+
|8914d64a9cbffff|POLYGON ((135.068...|POLYGON ((1503575...| POLYGON ((1503410...|POLYGON ((1503534...|  POLYGON ((1503493...|
|8914d64f1cfffff|POLYGON ((135.157...|POLYGON ((1504561...| POLYGON ((1504396...|POLYGON ((1504520...|  POLYGON ((1504479...|
|8914d64e573ffff|POLYGON ((135.170...|POLYGON ((1504716...| POLYGON ((1504551...|POLYGON ((1504674...|  POLYGON ((1504633...|
|8914d64ad7bffff|POLYGON ((135.056...|POLYGON ((1503444...| POLYGON ((1503278...|POLYGON ((1503402...|  POLYGON ((1503361...|
|8914d648abbffff|POLYGON ((135.106...|POLYGON ((1504002...| POLYGON ((1503837...|POLYGON ((1503961...|  POLYGON ((1503

In [33]:
# Анализ площади промышленных предприятий в радиусе 2 км
industrial_area_df = sedona.sql("""
    SELECT
        h3.h3_index,
        SUM(ST_Area(ST_Intersection(
            h3.industrial_buffer_2km,
            ST_Transform(i.geometry, 'EPSG:4326', 'EPSG:3857')
        ))) AS industrial_area_2km
    FROM h3_buffers h3
    LEFT JOIN industrial i ON ST_Intersects(
        h3.industrial_buffer_2km,
        ST_Transform(i.geometry, 'EPSG:4326', 'EPSG:3857')
    )
    GROUP BY h3.h3_index
""")
industrial_area_df.createOrReplaceTempView("h3_industrial_area")

In [34]:
# Проверка результатов
industrial_area_df.show()

+---------------+-------------------+
|       h3_index|industrial_area_2km|
+---------------+-------------------+
|8914d6483d3ffff|               NULL|
|8914d64113bffff| 2533856.4274095953|
|8914d648b4fffff| 3358330.2779846136|
|8914d649a2bffff| 40920.758064924725|
|8914d64ab27ffff|  877955.1322172302|
|8914d648d7bffff| 2348300.8902323395|
|8914d640acfffff|  52865.00105875887|
|8914d648e17ffff|  854579.3438874407|
|8914d649333ffff| 101953.44144670096|
|8914d64810fffff|               NULL|
|8914d64187bffff|  1802956.447835444|
|8914d648b43ffff|  2740563.164189819|
|8914d64f5b7ffff|  610307.9004423403|
|892eda4dd0bffff|               NULL|
|8914d641d5bffff|  554431.8822325215|
|8914d64e0c3ffff|   769301.578935798|
|8914d648bb7ffff|  910012.2310649039|
|8914d64a937ffff|   974078.788476434|
|892eda4cb23ffff|  48945.86851423237|
|8914d64883bffff|   4231556.89163529|
+---------------+-------------------+
only showing top 20 rows



In [35]:
# Анализ площади автомагистралей в радиусе 500 м
highway_area_df = sedona.sql("""
    SELECT
        h3.h3_index,
        SUM(ST_Area(ST_Intersection(
            h3.highway_buffer_500m,
            ST_Transform(h.geometry, 'EPSG:4326', 'EPSG:3857')
        ))) AS highway_area_500m
    FROM h3_buffers h3
    LEFT JOIN highways h ON ST_Intersects(
        h3.highway_buffer_500m,
        ST_Transform(h.geometry, 'EPSG:4326', 'EPSG:3857')
    )
    GROUP BY h3.h3_index
""")
highway_area_df.createOrReplaceTempView("h3_highway_area")

In [36]:
# Анализ площади зеленых зон и водоемов в радиусе 1 км
green_water_area_df = sedona.sql("""
    SELECT
        h3.h3_index,
        SUM(ST_Area(ST_Intersection(
            h3.green_water_buffer_1km,
            ST_Transform(g.geometry, 'EPSG:4326', 'EPSG:3857')
        ))) AS green_water_area_1km
    FROM h3_buffers h3
    LEFT JOIN (SELECT geometry FROM parks UNION SELECT geometry FROM water) g
    ON ST_Intersects(
        h3.green_water_buffer_1km,
        ST_Transform(g.geometry, 'EPSG:4326', 'EPSG:3857')
    )
    GROUP BY h3.h3_index
""")
green_water_area_df.createOrReplaceTempView("h3_green_water_area")

In [37]:
# Объединение всех метрик
h3_metrics_df = sedona.sql("""
    SELECT
        h.h3_index,
        h.geometry,
        COALESCE(i.industrial_area_2km, 0) AS industrial_area_2km,
        COALESCE(hw.highway_area_500m, 0) AS highway_area_500m,
        COALESCE(gw.green_water_area_1km, 0) AS green_water_area_1km
    FROM h3_buffers h
    LEFT JOIN h3_industrial_area i ON h.h3_index = i.h3_index
    LEFT JOIN h3_highway_area hw ON h.h3_index = hw.h3_index
    LEFT JOIN h3_green_water_area gw ON h.h3_index = gw.h3_index
""")
h3_metrics_df.createOrReplaceTempView("h3_metrics")


In [38]:
sedona.sql("SELECT COUNT(*) FROM h3_buffers").show()

+--------+
|count(1)|
+--------+
|    2976|
+--------+



In [39]:
# Нормализация метрик для H3-ячеек
from pyspark.ml.feature import MinMaxScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.functions import vector_to_array

# Подготовка данных для нормализациив
h3_metrics_df = h3_metrics_df.select(
    "h3_index",
    "geometry",
    col("industrial_area_2km").cast("double"),
    col("highway_area_500m").cast("double"),
    col("green_water_area_1km").cast("double")
)

h3_metrics_df.cache()
h3_metrics_df.printSchema()
# h3_metrics_df.show(5)

root
 |-- h3_index: string (nullable = true)
 |-- geometry: geometry (nullable = true)
 |-- industrial_area_2km: double (nullable = false)
 |-- highway_area_500m: double (nullable = false)
 |-- green_water_area_1km: double (nullable = false)



In [40]:
# Подготовка к нормализации
columns_to_normalize = ["industrial_area_2km", "highway_area_500m", "green_water_area_1km"]
assembler = VectorAssembler(inputCols=columns_to_normalize, outputCol="features")
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
pipeline = Pipeline(stages=[assembler, scaler])

# Применение нормализации
scaler_model = pipeline.fit(h3_metrics_df)
h3_scaled_df = scaler_model.transform(h3_metrics_df)

In [41]:
# Разворачиваем нормализованные значения в отдельные колонки
h3_final_df = h3_scaled_df.select(
    "h3_index",
    "geometry",
    vector_to_array("scaled_features").alias("scaled_array")
).select(
    "h3_index",
    "geometry",
    col("scaled_array")[0].alias("industrial_area_norm"),
    col("scaled_array")[1].alias("highway_area_norm"),
    col("scaled_array")[2].alias("green_water_area_norm")
).withColumn(
    "ecological_index",
    col("green_water_area_norm") - (
        col("industrial_area_norm") + col("highway_area_norm")
    )
)

h3_final_df.createOrReplaceTempView("h3_final")

In [42]:
# Визуализация результатов на карте
h3_choropleth_map = SedonaPyDeck.create_choropleth_map(
    df=h3_final_df,
    plot_col='ecological_index',
    map_style='light'
)
h3_choropleth_map

Output hidden; open in https://colab.research.google.com to view.

In [43]:
# Сохранение результатов в geojson
from pyspark.sql.functions import expr
def sedona_to_geojson(sedona_df, output_path, geometry_column='geometry'):
    # Преобразуем геометрию в GeoJSON формат
    geojson_df = sedona_df.withColumn(
        "temp_geojson", expr(f"ST_AsGeoJSON({geometry_column})")
    ).drop(geometry_column).withColumnRenamed("temp_geojson", "geometry")

    # Записываем как JSON (GeoJSON совместимый формат)
    geojson_df.coalesce(1).write.option("header", "false").mode("overwrite").json(output_path)

    print(f"GeoJSON сохранён в: {output_path}")

sedona_to_geojson(h3_final_df, 'h3_ecological_index.geojson')

GeoJSON сохранён в: h3_ecological_index.geojson


In [44]:
from pyspark.sql.functions import expr

def sedona_to_csv(sedona_df, output_path, geometry_column='geometry'):
    # Преобразуем геометрию в формат WKT
    wkt_df = sedona_df.withColumn(
        "temp_wkt", expr(f"ST_AsText({geometry_column})")
    ).drop(geometry_column).withColumnRenamed("temp_wkt", "geometry")

    # Сохраняем DataFrame как единый CSV файл
    wkt_df.coalesce(1).write.option("header", "true").mode("overwrite").csv(output_path)

    return f"Сохранено в {output_path}"

sedona_to_csv(h3_final_df, 'h3_ecological_index.csv')

'Сохранено в h3_ecological_index.csv'

#### **5. Применение методов машинного обучения для экологического зонирования**


- Выполните кластеризацию ячеек H3 по экологическим характеристикам
- Проведите детальный анализ полученных кластеров и составьте их экологические профили
- Обучите классификатор для определения экологического статуса новых территорий
- Оцените качество построенных моделей и выберите оптимальную
- Визуализируйте результаты экологического зонирования на карте города

In [45]:
# Кластеризация H3-полигонов по их характеристикам
from pyspark.ml.clustering import KMeans
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col

def cluster_h3_ecology(sedona_df, k=5):
    # Список признаков для кластеризации
    feature_columns = [
        "industrial_area_norm",
        "highway_area_norm",
        "green_water_area_norm",
        "ecological_index"
    ]

    # Преобразование строковых столбцов в числовой формат
    for column in feature_columns:
        sedona_df = sedona_df.withColumn(column, col(column).cast(DoubleType()))

    # Создание вектора признаков для алгоритма кластеризации
    assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

    # Преобразование данных
    h3_features_df = assembler.transform(sedona_df)

    # Создание и обучение модели KMeans
    kmeans = KMeans().setK(k).setSeed(42)
    model = kmeans.fit(h3_features_df)

    # Вывод центров кластеров
    centers = model.clusterCenters()
    print("Центры кластеров:")
    for i, center in enumerate(centers):
        print(f"Кластер {i}: {center}")

    # Применение модели для получения предсказаний
    clustered_df = model.transform(h3_features_df)

    return clustered_df, model

In [46]:
# Применение кластеризации
clustered_df, kmeans_model = cluster_h3_ecology(h3_final_df, k=5)

Центры кластеров:
Кластер 0: [0.07436764 0.00280447 0.41044139 0.33326927]
Кластер 1: [3.05144309e-02 3.64030682e-04 7.85363485e-01 7.54485024e-01]
Кластер 2: [ 0.17632759  0.61299229  0.1272907  -0.66202918]
Кластер 3: [ 0.07893351  0.00327922  0.02832786 -0.05388487]
Кластер 4: [ 0.41654813  0.00254932  0.01423288 -0.40486458]


In [47]:
clustered_df.show()

+---------------+--------------------+--------------------+------------------+---------------------+--------------------+--------------------+----------+
|       h3_index|            geometry|industrial_area_norm| highway_area_norm|green_water_area_norm|    ecological_index|            features|prediction|
+---------------+--------------------+--------------------+------------------+---------------------+--------------------+--------------------+----------+
|8914d64a9cbffff|POLYGON ((135.068...|0.001517439663260071|0.5650655051393698|  0.38821402131226757|-0.17836892349036237|[0.00151743966326...|         2|
|8914d64f1cfffff|POLYGON ((135.157...|   0.201013340454655|               0.0| 0.001301501245565782|-0.19971183920908922|[0.20101334045465...|         3|
|8914d64e573ffff|POLYGON ((135.170...| 0.06276945637501527|               0.0| 0.004105640523160098|-0.05866381585185517|[0.06276945637501...|         3|
|8914d64ad7bffff|POLYGON ((135.056...|                 0.0|               0.

In [48]:
# Анализ полученных кластеров
from pyspark.sql import functions as F

def analyze_clusters(clustered_df):
    # Удаление избыточного столбца features
    cleaned_df = clustered_df.drop("features")

    # Определение столбцов с признаками
    feature_cols = ["industrial_area_norm", "highway_area_norm",
                    "green_water_area_norm", "ecological_index"]

    # Распределение кластеров - количество и процент точек в каждом кластере
    total_count = cleaned_df.count()
    cluster_counts = cleaned_df.groupBy("prediction") \
                               .count() \
                               .withColumn("%", F.round(F.col("count") * 100 / total_count, 2)) \
                               .orderBy("prediction")
    cluster_counts.show()

    # Средние значения признаков по кластерам
    agg_exprs = [F.round(F.avg(col), 4).alias(col) for col in feature_cols]
    cluster_means = cleaned_df.groupBy("prediction") \
                              .agg(*agg_exprs) \
                              .orderBy("prediction")
    cluster_means.show()

    # Минимальные и максимальные значения для каждого признака по кластерам
    for feature in feature_cols:
        print(f"\nСтатистика для {feature}:")
        feature_stats = cleaned_df.groupBy("prediction") \
                                  .agg(
                                      F.round(F.min(feature), 4).alias("min"),
                                      F.round(F.max(feature), 4).alias("max")
                                  ) \
                                  .orderBy("prediction")
        feature_stats.show()

    return cleaned_df

In [49]:
cleaned_df = analyze_clusters(clustered_df)

+----------+-----+-----+
|prediction|count|    %|
+----------+-----+-----+
|         0|  424|14.25|
|         1|  293| 9.85|
|         2|   44| 1.48|
|         3| 1553|52.18|
|         4|  662|22.24|
+----------+-----+-----+

+----------+--------------------+-----------------+---------------------+----------------+
|prediction|industrial_area_norm|highway_area_norm|green_water_area_norm|ecological_index|
+----------+--------------------+-----------------+---------------------+----------------+
|         0|              0.0744|           0.0028|               0.4104|          0.3333|
|         1|              0.0305|           4.0E-4|               0.7854|          0.7545|
|         2|              0.1763|            0.613|               0.1273|          -0.662|
|         3|              0.0789|           0.0033|               0.0283|         -0.0539|
|         4|              0.4165|           0.0025|               0.0142|         -0.4049|
+----------+--------------------+-------------

In [50]:
# Обучение классификатора для определения экологического статуса новых территорий
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

def train_classification_models(cleaned_df, test_size=0.3, random_seed=42):
    """
    Обучает несколько моделей классификации и сравнивает их метрики
    """
    # Переименование столбца prediction в cluster_label для избежания конфликта
    cleaned_df = cleaned_df.withColumnRenamed("prediction", "cluster_label")

    # Определение признаков для обучения моделей
    feature_cols = ["industrial_area_norm", "highway_area_norm",
                    "green_water_area_norm", "ecological_index"]

    # Создание вектора признаков
    assembler = VectorAssembler(
        inputCols=feature_cols,
        outputCol="features"
    )

    # Разделение данных на обучающую и тестовую выборки
    train_df, test_df = cleaned_df.randomSplit([1 - test_size, test_size], seed=random_seed)

    print(f"Размер обучающей выборки: {train_df.count()}")
    print(f"Размер тестовой выборки: {test_df.count()}")

    # Создание словаря с моделями
    models = {
        "Логистическая регрессия": LogisticRegression(featuresCol="features", labelCol="cluster_label", maxIter=10),
        "Случайный лес": RandomForestClassifier(featuresCol="features", labelCol="cluster_label", numTrees=20),
        "Дерево решений": DecisionTreeClassifier(featuresCol="features", labelCol="cluster_label"),
    }

    # Создание объектов для оценки моделей
    evaluator_accuracy = MulticlassClassificationEvaluator(
        labelCol="cluster_label",
        predictionCol="prediction",
        metricName="accuracy"
    )

    evaluator_f1 = MulticlassClassificationEvaluator(
        labelCol="cluster_label",
        predictionCol="prediction",
        metricName="f1"
    )

    evaluator_precision = MulticlassClassificationEvaluator(
        labelCol="cluster_label",
        predictionCol="prediction",
        metricName="weightedPrecision"
    )

    evaluator_recall = MulticlassClassificationEvaluator(
        labelCol="cluster_label",
        predictionCol="prediction",
        metricName="weightedRecall"
    )

    # Создание схемы для Spark DataFrame с метриками
    schema = StructType([
        StructField("Модель", StringType(), False),
        StructField("Точность (Accuracy)", DoubleType(), False),
        StructField("F1-мера", DoubleType(), False),
        StructField("Точность (Precision)", DoubleType(), False),
        StructField("Полнота (Recall)", DoubleType(), False)
    ])

    # Список для хранения строк с результатами
    results_rows = []
    best_model = None
    best_f1 = -1

    # Обучение моделей и оценка их качества
    for name, model in models.items():
        print(f"\n#### Обучаем модель: {name}")

        # Создание pipeline с преобразованием признаков и моделью
        pipeline = Pipeline(stages=[assembler, model])

        # Обучение модели
        pipeline_model = pipeline.fit(train_df)

        # Предсказания на тестовой выборке
        predictions = pipeline_model.transform(test_df)

        # Оценка метрик
        accuracy = evaluator_accuracy.evaluate(predictions)
        f1 = evaluator_f1.evaluate(predictions)
        precision = evaluator_precision.evaluate(predictions)
        recall = evaluator_recall.evaluate(predictions)

        # Добавление результатов в список
        results_rows.append((name, accuracy, f1, precision, recall))

        print(f"Точность (Accuracy): {accuracy:.4f}")
        print(f"F1-мера: {f1:.4f}")
        print(f"Точность (Precision): {precision:.4f}")
        print(f"Полнота (Recall): {recall:.4f}")

        # Обновление лучшей модели по F1-мере
        if f1 > best_f1:
            best_model = pipeline_model
            best_f1 = f1

    # Создание Spark DataFrame с результатами
    spark = cleaned_df.sparkSession
    results_df = spark.createDataFrame(results_rows, schema)

    # Сортировка по F1-мере (в убывающем порядке)
    results_df = results_df.orderBy(F.col("F1-мера").desc())

    print("\n#### Сравнение моделей (отсортировано по F1-мере):")
    results_df.show(truncate=False)

    # Вывод названия лучшей модели
    best_model_name = results_df.select("Модель").first()[0]
    best_f1_value = results_df.select("F1-мера").first()[0]
    print(f"\n#### Лучшая модель: {best_model_name} с F1-мерой {best_f1_value:.4f}")

    return best_model, results_df

In [51]:
# Обучение моделей классификации
best_model, metrics_df = train_classification_models(cleaned_df)

Размер обучающей выборки: 2137
Размер тестовой выборки: 839

#### Обучаем модель: Логистическая регрессия
Точность (Accuracy): 0.9738
F1-мера: 0.9741
Точность (Precision): 0.9760
Полнота (Recall): 0.9738

#### Обучаем модель: Случайный лес
Точность (Accuracy): 0.9750
F1-мера: 0.9748
Точность (Precision): 0.9754
Полнота (Recall): 0.9750

#### Обучаем модель: Дерево решений
Точность (Accuracy): 0.9738
F1-мера: 0.9736
Точность (Precision): 0.9740
Полнота (Recall): 0.9738

#### Сравнение моделей (отсортировано по F1-мере):
+-----------------------+-------------------+------------------+--------------------+------------------+
|Модель                 |Точность (Accuracy)|F1-мера           |Точность (Precision)|Полнота (Recall)  |
+-----------------------+-------------------+------------------+--------------------+------------------+
|Случайный лес          |0.9749702026221693 |0.9747993278567271|0.975447541972551   |0.9749702026221692|
|Логистическая регрессия|0.9737783075089392 |0.97407746

In [52]:
# Тестирование модели на новых данных
def test_prediction(best_model, cleaned_df, features_to_predict):
    """
    Выполняет тестовое предсказание с использованием лучшей модели
    """
    # Проверка наличия столбца cluster_label
    if "cluster_label" not in cleaned_df.columns:
        # Если нет, переименовываем prediction в cluster_label
        cleaned_df = cleaned_df.withColumnRenamed("prediction", "cluster_label")

    # Определение признаков
    feature_cols = ["industrial_area_norm", "highway_area_norm",
                    "green_water_area_norm", "ecological_index"]

    # Получение сессии Spark из входного DataFrame
    spark = cleaned_df.sparkSession

    # Создание тестового DataFrame
    test_row = spark.createDataFrame([features_to_predict], feature_cols)

    # Выполнение предсказания
    prediction = best_model.transform(test_row)

    # Получение предсказанного кластера
    predicted_cluster = prediction.select("prediction").collect()[0][0]

    print(f"#### Тестовое предсказание:")
    print(f"Признаки: {features_to_predict}")
    print(f"Предсказанный кластер: {predicted_cluster}")

    # Анализ характеристик этого кластера
    cluster_stats = cleaned_df.filter(f"cluster_label = {predicted_cluster}") \
                               .agg(*[F.avg(col).alias(col) for col in feature_cols])

    print("\n#### Характеристики предсказанного кластера:")
    cluster_stats.show()

    return predicted_cluster

In [53]:
# Тестирование модели на новых данных
test_features = [0.5, 0.8, 0.6, 0.4]
predicted_cluster = test_prediction(best_model, cleaned_df, test_features)

#### Тестовое предсказание:
Признаки: [0.5, 0.8, 0.6, 0.4]
Предсказанный кластер: 0.0

#### Характеристики предсказанного кластера:
+--------------------+--------------------+---------------------+------------------+
|industrial_area_norm|   highway_area_norm|green_water_area_norm|  ecological_index|
+--------------------+--------------------+---------------------+------------------+
| 0.07436764419587928|0.002804473069579592|   0.4104413917935438|0.3332692745280849|
+--------------------+--------------------+---------------------+------------------+



In [54]:
# Визуализация результатов кластеризации
cluster_map = SedonaPyDeck.create_choropleth_map(
    df=clustered_df.drop('features'),
    plot_col='prediction',
    map_style='light'
)
cluster_map

Output hidden; open in https://colab.research.google.com to view.

### **Требования к отчету**


1. Описание выбранной территории и источников данных
2. Методика расчета экологических показателей с обоснованием выбора весовых коэффициентов
3. Карты распределения отдельных экологических факторов и интегрального индекса
4. Результаты кластерного анализа с интерпретацией экологических профилей кластеров
5. Оценка точности классификационной модели и рекомендации по ее применению
6. Выводы об экологическом состоянии исследуемой территории и возможных мерах по его улучшению

**ИТОГ**

**1. Описание выбранной территории и источников данных**

В качестве территории для анализа выбраны районы Хабаровска:

1) Железнодорожный район. Район с развитой транспортной инфраструктурой — здесь находится железнодорожный вокзал Хабаровска. Также известен промышленными предприятиями и жилыми кварталами.

2) Индустриальный район. Один из крупнейших районов города по площади. Здесь сосредоточены промышленные зоны, логистические центры, а также жилые микрорайоны.

3) Кировский район. Расположен в северной части Хабаровска. Известен своими зелёными зонами и парками. В районе активно развивается жилое строительство.

4) Краснофлотский район. Находится вдоль берега Амура. Считается одним из наиболее живописных районов города. Здесь расположены набережные, зоны отдыха и предприятия судостроения.

5) Центральный район. Исторический и административный центр Хабаровска. Здесь расположены правительственные здания, музеи, театры, главные городские площади и туристические достопримечательности.



**2. Методика расчета экологических показателей с обоснованием выбора весовых коэффициентов**

Интегральный индекс экологического благополучия рассчитывался с помощью взвешенной формулы. Значения факторов предварительно нормализованы для сбалансированного вклада в общий результат. Весовые коэффициенты отражают значимость каждого фактора для оценки состояния территории.

**3. Карты распределения отдельных экологических факторов и интегрального индекса** представлены в тексте кода

- Положительно влияющие показатели:

green_area_norm — Процент покрытия зелеными насаждениями (вес: 0.4)

water_length_norm — Протяженность водных объектов (вес: 0.3)

- Отрицательно влияющие показатели:

pollution_density_norm — Количество и плотность источников загрязнения (вес: 0.2)

highways_density_norm — Плотность автомагистралей (вес: 0.1)

In [55]:
eco_index_map

<IPython.core.display.Javascript object>

{
  "initialViewState": {
    "latitude": 48.500004419999996,
    "longitude": 134.95356856,
    "zoom": 11
  },
  "layers": [
    {
      "@@type": "GeoJsonLayer",
      "autoHighlight": true,
      "data": [
        {
          "coordinate_array_sedona": [
            134.9803613,
            48.5765601
          ],
          "eco_index": 0.437056925456504,
          "geometry": {
            "coordinates": [
              [
                [
                  134.9803613,
                  48.5765601
                ],
                [
                  134.9805541,
                  48.5766783
                ],
                [
                  134.9814524,
                  48.5772868
                ],
                [
                  134.9822132,
                  48.5779796
                ],
                [
                  134.9831612,
                  48.5786301
                ],
                [
                  134.9838384,
                  48.5789464
                ],
                [
                  134.9847972,
                  48.5793648
                ],
                [
                  134.9858916,
                  48.5798723
                ],
                [
                  134.9869859,
                  48.5803266
                ],
                [
                  134.9880993,
                  48.5807766
                ],
                [
                  134.9892826,
                  48.5813231
                ],
                [
                  134.9904146,
                  48.5819634
                ],
                [
                  134.9913346,
                  48.5822788
                ],
                [
                  134.9930423,
                  48.582644
                ],
                [
                  134.9964588,
                  48.5831939
                ],
                [
                  134.9998758,
                  48.5837405
                ],
                [
                  135.0029369,
                  48.5842336
                ],
                [
                  135.0068695,
                  48.5848679
                ],
                [
                  135.0091423,
                  48.5852397
                ],
                [
                  135.0111855,
                  48.5855732
                ],
                [
                  135.0142354,
                  48.5860697
                ],
                [
                  135.0168263,
                  48.586492
                ],
                [
                  135.0198452,
                  48.586985
                ],
                [
                  135.0233807,
                  48.587568
                ],
                [
                  135.025685,
                  48.5879505
                ],
                [
                  135.0278242,
                  48.588305
                ],
                [
                  135.0297733,
                  48.5886275
                ],
                [
                  135.0308273,
                  48.5888016
                ],
                [
                  135.0315353,
                  48.5889631
                ],
                [
                  135.0315401,
                  48.5890695
                ],
                [
                  135.031573,
                  48.589179
                ],
                [
                  135.0316031,
                  48.5892762
                ],
                [
                  135.0316178,
                  48.5892913
                ],
                [
                  135.0316329,
                  48.5893013
                ],
                [
                  135.0316546,
                  48.5893493
                ],
                [
                  135.0317132,
                  48.5893453
                ],
                [
                  135.0317771,
                  48.5894955
       

**4. Результаты кластерного анализа с интерпретацией экологических профилей кластеров**

В ходе кластерного анализа были выделены 5 кластеров, каждый из которых характеризуется определенным экологическим состоянием:

**Кластер 0:** высокий уровень загрязнения и низкое количество зеленых зон. Средний экологический показатель - **плохая экология**.

**Кластер 1:** преобладание зеленых насаждений, однако имеются небольшие промышленные зоны. Средний экологический показатель - **приемлемый уровень экологии**.

**Кластер 2:** высокая концентрация промышленности и минимальное количество зеленых насаждений. Средний экологический показатель - **очень плохая экология**.

**Кластер 3:** смешанные зоны с умеренным количеством промышленных зон и наличием зеленых территорий, но высоким уровнем загрязнения от дорог.Средний экологический показатель - **сниженный уровень экологии**.

**Кластер 4:** зоны с низким уровнем загрязнения и большими озелененными территориями. Средний экологический показатель - **хорошая экология**.

**5. Оценка точности классификационной модели и рекомендации по ее применению**

Для классификации кластеров использовались три модели:
- **логистическая регрессия** точность 97.3%, F1-мера 0.97407,
- **случайный лес** точность 97.5%, F1-мера 0.9747,
- **дерево решений** точность 97.3%, F1-мера 0.9736.


**6. Выводы об экологическом состоянии исследуемой территории и возможных мерах по его улучшению**

1. **Промышленные и проблемные зоны (кластеры 0, 2, 3):**  
   - Увеличить озеленение (парки, аллеи, дворы, детские площадки).  
   - Снизить выбросы (очистные сооружения, шумозащитные экраны, посадка деревьев вдоль дорог).  

2. **Территории с умеренным уровнем (кластер 1):**  
   - Расширить зелёные зоны, обустроить новые парки и рекреационные пространства.  

3. **Благополучные экозоны (кластер 4):**  
   - Сохранять существующие зелёные насаждения и водоёмы, предотвращая их деградацию.  

